In [1]:
import pyximport
pyximport.install()
import fast_tools

In [2]:
import numpy as np
import sharedmem
from multiprocessing import Pool
from functools import partial
import sys

def stuff_to_do_extern(process, W_shared):
        print(f"My process is {process}")

        sum = 0.0
        for i in range(W_shared.shape[0]):
            sum += W_shared[i, 0]
            
        print(f"My process was {process}")
            
        return sum
    
def stuff_to_do_extern_cython(process, W_shared):
        print(f"My process is {process}")
        res = fast_tools.col_0_sum(W_shared)
            
        print(f"My process was {process} and my result is {res}")
            
        return res

class test_class():

    def __init__(self, test_size):
        self.W = np.full((test_size, test_size), 1.0, dtype=np.float32)
        self.W_shared = sharedmem.empty(self.W.shape, dtype=np.float32)
        self.W_shared[:, :] = self.W[:, :]
        
    def stuff_to_do(self, process):
        print(f"My process ist {process}")
        sum = 0.0
        for i in range(self.W.shape[0]):
            sum += self.W[i, 0]
            
        return sum
    
    def par_func(self, cores):
        if cores == 1:
            res = self.stuff_to_do(1)
            return res
        else:
            pool = Pool(cores)
            partial_stuff_to_do = partial(self.stuff_to_do)
            
            hits = np.sum(np.array((pool.map(partial_stuff_to_do,(process for process in range(cores))))))
            pool.close()
            pool.join()
            return hits / cores
        
    def par_func2(self, cores):
        if cores == 1:
            res = self.stuff_to_do(1)
            return res
        else:
            pool = Pool(cores)
            partial_stuff_to_do = partial(stuff_to_do_extern, W_shared=self.W_shared)
            
            hits = np.sum(np.array((pool.map(partial_stuff_to_do,(process for process in range(cores))))))
            pool.close()
            pool.join()
            return hits / cores
        
    def par_func3(self, cores):
        if cores == 1:
            res = self.stuff_to_do(1)
            return res
        else:
            pool = Pool(cores)
            partial_stuff_to_do = partial(stuff_to_do_extern_cython, W_shared=self.W_shared)

            hits = np.sum(np.array((pool.map(partial_stuff_to_do,(process for process in range(cores))))))
            pool.close()
            pool.join()
            return hits / cores

In [3]:
test = test_class(24000)

In [4]:
test.par_func3(cores=8)

My process is 1
My process is 2
My process is 0
My process is 6
My process is 3
My process is 5
My process is 7
My process is 4
My process was 7 and my result is 24000.0
My process was 6 and my result is 24000.0
My process was 3 and my result is 24000.0
My process was 0 and my result is 24000.0
My process was 2 and my result is 24000.0
My process was 1 and my result is 24000.0
My process was 4 and my result is 24000.0
My process was 5 and my result is 24000.0


24000.0

In [5]:
test.par_func(cores=8)

error: 'i' format requires -2147483648 <= number <= 2147483647

error: 'i' format requires -2147483648 <= number <= 2147483647

In [14]:
test.par_func2(cores=8)

My process is 0
My process was 0
My process is 1
My process was 1
My process is 2
My process was 2
My process is 3
My process was 3
My process is 4
My process was 4
My process is 5
My process was 5
My process is 6
My process was 6
My process is 7
My process was 7


20000.0

In [15]:
test.par_func(cores=8)

My process ist 0
My process ist 1
My process ist 2
My process ist 3
My process ist 4
My process ist 5
My process ist 6
My process ist 7


20000.0

In [3]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size